In [4]:
# I chose the Exponential Smoothing (Holt-Winters) model


import os
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing


# 1) Load data
try:
    HERE = os.path.dirname(__file__)
except NameError:
    HERE = os.getcwd()  # Jupyter/Colab fallback

TRAIN_PATH = os.path.join(HERE, "assignment_data_train.csv")
TEST_PATH  = os.path.join(HERE, "assignment_data_test.csv")

if os.path.exists(TRAIN_PATH) and os.path.exists(TEST_PATH):
    train = pd.read_csv(TRAIN_PATH, parse_dates=["Timestamp"])
    test  = pd.read_csv(TEST_PATH,  parse_dates=["Timestamp"])
else:

    TRAIN_PATH = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
    TEST_PATH  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"
    train = pd.read_csv(TRAIN_PATH, parse_dates=["Timestamp"])
    test  = pd.read_csv(TEST_PATH,  parse_dates=["Timestamp"])


train = train.sort_values("Timestamp").set_index("Timestamp")
y = pd.to_numeric(train["trips"], errors="coerce")


# 2) Define algorithm object and fit


model = ExponentialSmoothing(y, trend="add", seasonal="add", seasonal_periods=24)
modelFit = model.fit(optimized=True)


# 3) Forecast
h = len(test)
_fc = modelFit.forecast(steps=h)
pred = pd.Series(_fc, index=pd.to_datetime(test["Timestamp"]))


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
